In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')

In [4]:
model = GPT2LMHeadModel.from_pretrained('gpt2-large', pad_token_id=tokenizer.eos_token_id)
model.to(device);

In [30]:
import re

stop_pattern = re.compile(r'[A-Z]:|\n\n')
end_punc = {'.', '!', '?'}


def cleanse_answer(answer):
    match = stop_pattern.search(answer)
    stop_position = match.start() if match else len(answer)
    answer = answer[:stop_position]
    if set(answer) & end_punc:
        end_index = len(answer) - 1
        while answer[end_index] not in end_punc:
            end_index -= 1
        answer = answer[:end_index + 1]
    return answer.strip()


def decode_answer(model_output, input_text_len):
    answer = tokenizer.decode(model_output, skip_special_tokens=True)[input_text_len:]
    return cleanse_answer(answer)


def print_samples(question, top_p=0.5, top_k=None, temperature=0.9, num_beams=3,
            repetition_penalty=None, no_repeat_ngram_size=4, num_return_sequences=1, repeat=3):
    input_text = f'Q: {question} A:'
    input_ids = tokenizer.encode(input_text)
    input_ids = torch.tensor([input_ids]).to(device)
    for _ in range(repeat):
        sample_outputs = model.generate(
            input_ids,
            do_sample=True, 
            max_length=len(input_ids) + 175, 
            top_p=top_p, 
            top_k=top_k, 
            temperature=temperature,
            num_beams=num_beams,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            num_return_sequences=num_return_sequences,
#             length_penalty=2.0
        )
        answers = sorted(map(lambda text: decode_answer(text, len(input_text)), sample_outputs),
                         key=len, reverse=True)
        for answer in answers:
            print(answer)
            print('**********\n')

In [31]:
print_samples("""What's the grossest thing you've ever tasted?""")

I don't know. I don't think I've ever tasted anything gross.
**********

I don't know. I've never tasted anything that was gross.
**********

I don't know. I don't think I've ever tasted anything gross.
**********



In [32]:
print_samples("""If you could have 1 superpower what would it be and why?""")

I would like to see a world where everyone has access to clean water, clean air, clean food, and clean energy. I would also like to see everyone have access to a decent education. I would like everyone to have access to health care. I want everyone to be able to have a decent standard of living. I want to see everyone be able to live in a safe and secure environment.
**********

I would like to see a world where there are no superpowers. I think that would be a good thing.
**********

I would like to see a world where there are no superpowers. I think that would be a good thing. I think the world would be a better place if there were no superpowers, and I think that's a good thing for the world.
**********



In [33]:
print_samples("""Is there anything you should be doing right now, but you're wasting your time on reddit instead? If so, what is it?""")

I don't think I should be doing anything right now. I should be spending time with my family. I don't know what to do with my life right now, so I'm just wasting my time on reddit.
**********

I'm not sure what I should be doing. I've been doing a lot of work on the game, and I've been working on it for a long time. I don't know if I should be working on reddit or not. I'm not really sure what to do. I'm trying to figure it out.
**********

I'm not sure what I should be doing, but I'm not wasting my time on reddit either. I've been doing a lot of work on my website, and I've been working on a lot of other things as well. I don't have a lot of free time right now, so I don't know if I should be working on reddit or not. I'm not going to lie, I'm a little nervous about it. I'm going to try to do as much as I can, but I don't think I'll be able to do everything I want to do.
**********



In [34]:
print_samples("""What is the scariest/creepiest theory you know about?""")

I don't know. I think it's a good question. I don't think it's scary at all. It's just a theory.
**********

I don't know. I think it's a good question. I don't think it's scary. It's just a theory.
**********

I don't know if it's scary or creepy, but I think it's a good theory. It's a theory that I've been thinking about for a long time, and it's one that I think has a lot of merit. It's the idea that the universe is a big game of hide-and-seek, and we're all trying to find our way out of it.
**********



In [35]:
print_samples("""In the John Wick universe, assassins are shown to be everyone and everywhere, thus indicating a buyer's market. What limited time deals and offers do assassins provide to out price the competition? How does the economics of the world operate with so many trained killers?""")

I don't think it's limited time deals or offers. I think it's the fact that there are so many of them. There are so many people who want to be a part of this world, and they're willing to do whatever it takes to get there. They're willing to kill for it. They're going to do anything to get there, and they'll do it for the right price. I think that's the reality of the world, and that's why I think the world is so dangerous. It's because there's so many people willing to do anything for it.
**********

The economics of the market is a very complex subject, and I don't want to get into too much detail here, but suffice it to say that there are a number of factors at play. First, there is the fact that there are so many people out there who want to kill you. Second, there are a lot of people who are willing to kill you for a lot of money. Third, there is a lot of competition in the market for the services of assassins. Fourth, there are many different types of assassins, and each type has

In [36]:
print_samples("""What is you favorite sex scene in the Bible?""")

I'm a big fan of the story of Sodom and Gomorrah. I think it's one of the best stories in the Bible. It's a story of a city that's destroyed by God, and it's a story about God's wrath against the people of Sodom. It's also a story of God's love for the people of the city. It's not just a story about the destruction of the city, but it's also about God's love and mercy for the people.
**********

I have a favorite sex scene from the Bible. It's in the book of Leviticus. Leviticus 19:22 says, "You shall not lie with a male as one lies with a woman; it is an abomination." And it says, "If a man lies with a man as with a woman, both of them have committed an abomination; they shall surely be put to death; their blood shall be upon them." And it goes on to say, "If you lie with a woman as with a man, both of you have done what is detestable. They shall surely be cut off from among their people.
**********

I think the best sex scene is in the book of Leviticus. I think it's one of the best 

In [37]:
print_samples("""What are underrated websites and what do you use them for?""")

I use them for a lot of different things. I use them to keep up with the news, to keep track of what's going on in the world, to keep in touch with my friends and family, to keep up on what's happening in the world. I also use them to stay up to date with what's happening with my favorite bands and artists.
**********

I use a lot of websites. I'm a big fan of The Huffington Post. I use it for a lot of different things. I also use it for my personal blog. I'm also a huge fan of The New York Times. I use that a lot.
**********

I use a lot of websites for different things. For example, I have a website for my blog and I use it for a lot of different things. I also use a website for a book I'm writing. I use a website to keep track of my blog posts. I also have a website that I use for a book that I'm writing and I use that website for that book. I use websites for a lot different things.
**********



In [38]:
print_samples("""Why do you hate the people that you hate?""")

Because they are the ones who are trying to kill us.
**********

Because they are the ones that are going to get us into trouble. They are the ones who are going to make us look bad. They are going to be the ones that make us look weak. They are not going to be our friends. They will be our enemies.
**********

Because they are the ones that are going to kill you.
**********



In [39]:
print_samples("""People who used to work in photo labs before digital cameras, what weird stuff did you develop?""")

I developed a lot of stuff that I don't think I would have been able to develop if I hadn't been in the photo lab. I developed a bunch of stuff that was never going to be used in a photo lab, and I developed a ton of stuff that would never have been used in a commercial photo lab.
**********

I developed a lot of weird stuff. I used to work at a photo lab in New York, and I used to develop a lot of stuff. I did a lot of color correction, a lot of black and white, and a lot of things like that.
**********

I developed a lot of weird stuff. I used to work at a photo lab, and there was a lot of stuff that I developed. I developed a bunch of stuff that was really weird. I developed some stuff that was like, I don't know what it is, but it was like, it was like a black and white film. It was like a color film. I don't even know what it was. It was really weird stuff.
**********



In [40]:
print_samples("""To the people who check behind the shower curtain before using the washroom; what’s the next plan if you see someone?""")

If you see someone in the shower, don't touch them. If you see them in the bathroom, don't use the washroom.
**********

If you see someone in the shower, don't touch them. If you see them in the bathroom, don't use the washroom. If you don't see anyone in the shower or bathroom, leave the area.
**********

If you see someone who looks like they’re about to use the washroom, you should leave them alone. If you’re in the shower and someone looks like they're about to use it, leave them alone, too. If you see a person who looks like he’s about to use a washroom, don’t let them use it.
**********



In [41]:
print_samples("""If you came with a warning label, what would it be?""")

If I had a warning label on my car, I would have a warning on my car.
**********

I don't know. I don't think I'd be able to tell you.
**********

I would say, "Please don't do this.
**********



In [42]:
print_samples("""What is the best decision you've made while drunk?""")

I don't know. I've never made a bad decision while drunk. I've made a lot of good decisions while drunk.
**********

I don't know. I don't think I've ever made a bad decision while drunk. I think I've made a lot of good decisions while drunk.
**********

I've made a lot of bad decisions while drunk.
**********



In [43]:
print_samples("""What’s one rule you live by?""")

I live by the fact that I don't take anything for granted.
**********

I don't live by one rule. I live by the fact that I'm going to do what I want to do. I don't care what anyone else thinks about it. I'm not going to listen to anybody else's opinion. I'm just going to do it.
**********

I live by the fact that I don't care what anyone thinks about me.
**********

